# Ejercicios

Los datos adjuntos a esta lección forman parte de la base de datos Football Data from Transfermarkt de Kaggle. El conjunto de datos se compone de varios archivos CSV con información sobre competiciones, juegos, clubes, jugadores y apariciones.

1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.
4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

Recursos de esta clase
data.zip

In [ ]:
# Importar librerías
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

# Crea una sesión de Spark
spark = SparkSession.builder.appName("EjercicioDataFrames").getOrCreate()

# 1. Determinar los tres países con mayor número de jugadores nacidos en ese país
players_df = spark.read.csv("players.csv", header=True, inferSchema=True)
top_countries = players_df.groupBy("country_of_birth").count().orderBy(desc("count")).limit(3)

# Muestra el resultado
top_countries.show()

# 2. Obtener la lista de jugadores con tarjeta roja
appearances_df = spark.read.csv("appearances.csv", header=True, inferSchema=True)
red_cards_df = appearances_df.groupBy("player_id").sum("red_cards").filter(col("sum(red_cards)") > 0)

# Une los DataFrames para obtener el nombre del jugador
red_cards_players_df = red_cards_df.join(players_df, "player_id").select("pretty_name", "sum(red_cards)")

# Renombra la columna sum(red_cards)
red_cards_players_df = red_cards_players_df.withColumnRenamed("sum(red_cards)", "red_cards_count")

# Muestra el resultado
red_cards_players_df.show()

# 3. Contar la cantidad de juegos en la Premier League
games_df = spark.read.csv("games.csv", header=True, inferSchema=True)
premier_league_games_count = games_df.filter(games_df.competition_code == "PL").count()

# Muestra el resultado
print("Cantidad de juegos en la Premier League:", premier_league_games_count)

# 4. Obtener las tres ligas con mayor asistencia de público
leagues_df = spark.read.csv("leagues.csv", header=True, inferSchema=True)
games_with_attendance_df = games_df.filter(col("attendance").isNotNull())
league_attendance = games_with_attendance_df.join(leagues_df, games_with_attendance_df.competition_code == leagues_df.league_id) \
    .groupBy("name").sum("attendance").orderBy(desc("sum(attendance)")).limit(3)

# Renombra la columna sum(attendance)
league_attendance = league_attendance.withColumnRenamed("sum(attendance)", "total_attendance")

# Muestra el resultado
league_attendance.show()

# Detiene la sesión de Spark
spark.stop()
